In [ ]:
import cProfile
import numpy as np
%matplotlib inline

In [ ]:
def to_be_profiled(n):
    if n != 0:
        return n * to_be_profiled(n-1)
    else:
        return 1

In [ ]:
%prun to_be_profiled(9)

In [ ]:
cProfile.run('to_be_profiled(9)')

In [ ]:
# Ex 0.6
A = np.arange(100)
C = np.cos(A) 
print C.ptp()

In [ ]:
# Ex 0.7
i = np.linspace(0, 0.999, 1000)
np.sum((i ** 2)/1000)

In [ ]:
# Ex 0.10
import lxmls.readers.galton as galton
galton_data = galton.load()

In [ ]:
# 
print "Father avg {0}".format(galton_data[:,0].mean())
print "Son avg {0}".format(galton_data[:,1].mean())

print "Father std {0}".format(galton_data[:,0].std())
print "Son std {0}".format(galton_data[:,1].std())

In [ ]:
plt.hist(galton_data[:,0].flatten(), bins=20, alpha=0.3)
plt.hist(galton_data[:,1].flatten(), bins=20, alpha=0.3)

plt.show()

In [ ]:
plt.plot(galton_data[:,0],galton_data[:,1], 'ro')
plt.xticks(xrange(60,80))
plt.yticks(xrange(60,80))
plt.show()

In [ ]:
randomized_data = galton_data + np.random.randn(galton_data.shape[0],2)

plt.plot(randomized_data[:,0],randomized_data[:,1], 'ro')



In [ ]:
# Ex 0.12
def get_y(x):
    return (x+2)**2 - 16*np.exp(-((x-2)**2))

def get_grad(x):
    return (2*x+4)-16*(-2*x + 4)*np.exp(-((x-2)**2))

In [ ]:
x = np.arange(-8,8,0.001)
y = get_y(x)
plt.plot(x,y, 'r')
plt.plot(x,get_grad(x),'b')


In [ ]:
def gradient_descent(start_x,func,grad, step): # Precision of the solution
    prec = 0.1
    #Use a fixed small step size 
    step_size = step
    alpha = 0.9
    #max iterations
    max_iter = 100
    x_new = start_x
    res = []
    for i in xrange(max_iter):
        x_old = x_new
        #Use beta egual to -1 for gradient descent
        x_new = x_old - step_size * grad(x_new) 
        f_x_new = func(x_new)
        f_x_old = func(x_old) 
        res.append([x_new,f_x_new]) 
        if(abs(f_x_new - f_x_old) < prec):
            print "change in function values too small, leaving"
            return np.array(res)
        step_size *= alpha 
    print "exceeded maximum number of iterations, leaving" 
    return np.array(res)

In [ ]:
x_0 = 8
res = gradient_descent(x_0,get_y,get_grad)
plt.plot(res[:,0],res[:,1],'+')
plt.plot(x,y, 'r')
plt.show()

In [ ]:
w = np.array([1,1])
y = galton_data[:,1]
x = np.vstack([galton_data[:,0], np.ones(len(galton_data[:,0]))]).T

def error(w):
    y_est = x.dot(w)
    return np.sum((y_est-y)**2)

def error_grad(w):
    return sum(2*(x.dot(w)).dot(x))

In [ ]:
np.linalg.lstsq(x, y)

In [ ]:
w

In [ ]:
w = [0.643,23.99]
error_grad(w)
2*(x.dot(w)).dot(x)

In [ ]:
gradient_descent([1,1],error, error_grad, 0.00000001)

# Day 2

In [ ]:
import cProfile
import numpy as np
%matplotlib inline

In [ ]:
import lxmls.readers.sentiment_reader as srs 
scr = srs.SentimentCorpus("books")

In [ ]:
import lxmls.classifiers.multinomial_naive_bayes as mnbb

mnb = mnbb.MultinomialNaiveBayes()
params_nb_sc = mnb.train(scr.train_X,scr.train_y)
y_pred_train = mnb.test(scr.train_X,params_nb_sc)
acc_train = mnb.evaluate(scr.train_y, y_pred_train)
y_pred_test = mnb.test(scr.test_X,params_nb_sc)
acc_test = mnb.evaluate(scr.test_y, y_pred_test)

print "Multinomial Naive Bayes Amazon Sentiment Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
import lxmls.readers.simple_data_set as sds
sd = sds.SimpleDataSet(nr_examples=100, g1 = [[-1,-1],1], g2 = [[1,1],1], balance=0.5, split=[0.5,0,0.5])

In [ ]:
import lxmls.classifiers.perceptron as percc
perc = percc.Perceptron(nr_epochs=21)
params_perc_sd, w_lst = perc.train(sd.train_X,sd.train_y)
y_pred_train = perc.test(sd.train_X,params_perc_sd)
acc_train = perc.evaluate(sd.train_y, y_pred_train)
y_pred_test = perc.test(sd.test_X,params_perc_sd)
acc_test = perc.evaluate(sd.test_y, y_pred_test)
print "Perceptron Simple Dataset Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
fig,axis = sd.plot_data()
fig,axis = sd.add_line(fig,axis,params_perc_sd,"Perceptron","blue")
for i in xrange(len(w_lst)):
    if i % 5 == 0:
        fig,axis = sd.add_line(fig,axis,params_perc_sd,"Percept %d" % (i),"blue")


In [ ]:
import lxmls.classifiers.perceptron as percc
perc = percc.Perceptron(nr_epochs=20)
params_perc_scr, w_lst = perc.train(scr.train_X,scr.train_y)
y_pred_train = perc.test(scr.train_X,params_perc_scr)
acc_train = perc.evaluate(scr.train_y, y_pred_train)
y_pred_test = perc.test(scr.test_X,params_perc_scr)
acc_test = perc.evaluate(scr.test_y, y_pred_test)
print "Perceptron Amazon Sentient Dataset Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
import lxmls.classifiers.mira as mirac
mira = mirac.Mira()
mira.regularizer = 0.5 # This is lambda
params_mira_sd = mira.train(sd.train_X,sd.train_y)
y_pred_train = mira.test(sd.train_X,params_mira_sd)
acc_train = mira.evaluate(sd.train_y, y_pred_train)
y_pred_test = mira.test(sd.test_X,params_mira_sd)
acc_test = mira.evaluate(sd.test_y, y_pred_test)
print "Mira Simple Dataset Accuracy train: %f test: %f"%(acc_train,acc_test) 
fig,axis = sd.plot_data()
fig,axis = sd.add_line(fig,axis,params_mira_sd,"Mira","green")
params_mira_sc = mira.train(scr.train_X,scr.train_y)
y_pred_train = mira.test(scr.train_X,params_mira_sc)
acc_train = mira.evaluate(scr.train_y, y_pred_train)
y_pred_test = mira.test(scr.test_X,params_mira_sc)
acc_test = mira.evaluate(scr.test_y, y_pred_test)
print "Mira Amazon Sentiment Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
import lxmls.classifiers.max_ent_batch as mebc
me_lbfgs = mebc.MaxEntBatch()
me_lbfgs.regularizer = 1.0
params_meb_sd = me_lbfgs.train(sd.train_X,sd.train_y)
y_pred_train = me_lbfgs.test(sd.train_X,params_meb_sd)
acc_train = me_lbfgs.evaluate(sd.train_y, y_pred_train)
y_pred_test = me_lbfgs.test(sd.test_X,params_meb_sd)
acc_test = me_lbfgs.evaluate(sd.test_y, y_pred_test)
print "Max-Ent batch Simple Dataset Accuracy train: %f test: %f"%(acc_train,acc_test
    )
fig,axis = sd.plot_data()
fig,axis = sd.add_line(fig,axis,params_meb_sd,"Max-Ent-Batch","orange")

In [ ]:
params_meb_sc = me_lbfgs.train(scr.train_X,scr.train_y)
y_pred_train = me_lbfgs.test(scr.train_X,params_meb_sc)
acc_train = me_lbfgs.evaluate(scr.train_y, y_pred_train)
y_pred_test = me_lbfgs.test(scr.test_X,params_meb_sc)
acc_test = me_lbfgs.evaluate(scr.test_y, y_pred_test)
print "Max-Ent Batch Amazon Sentiment Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
import lxmls.classifiers.max_ent_online as meoc
me_sgd = meoc.MaxEntOnline()
me_sgd.regularizer = 1.0
params_meo_sc = me_sgd.train(scr.train_X,scr.train_y)
y_pred_train = me_sgd.test(scr.train_X,params_meo_sc)
acc_train = me_sgd.evaluate(scr.train_y, y_pred_train)
y_pred_test = me_sgd.test(scr.test_X,params_meo_sc)
acc_test = me_sgd.evaluate(scr.test_y, y_pred_test)
print "Max-Ent Online Amazon Sentiment Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
import lxmls.classifiers.svm as svmc
svm = svmc.SVM()
svm.regularizer = 1.0 # This is lambda
params_svm_sd = svm.train(sd.train_X,sd.train_y)
y_pred_train = svm.test(sd.train_X,params_svm_sd)
acc_train = svm.evaluate(sd.train_y, y_pred_train)
y_pred_test = svm.test(sd.test_X,params_svm_sd)
acc_test = svm.evaluate(sd.test_y, y_pred_test)


print "SVM Online Simple Dataset Accuracy train: %f test: %f"%(acc_train,acc_test)
fig,axis = sd.add_line(fig,axis,params_svm_sd,"SVM","orange")
params_svm_sc = svm.train(scr.train_X,scr.train_y)
y_pred_train = svm.test(scr.train_X,params_svm_sc)
acc_train = svm.evaluate(scr.train_y, y_pred_train)
y_pred_test = svm.test(scr.test_X,params_svm_sc)
acc_test = svm.evaluate(scr.test_y, y_pred_test)
print "SVM Online Amazon Sentiment Accuracy train: %f test: %f"%(acc_train,acc_test)

In [ ]:
from  lxmls.run_all_classifiers import run_all_classifiers

run_all_classifiers(sd)

# Day 3 

In [ ]:
import cProfile
import numpy as np
%matplotlib inline

In [ ]:
import lxmls.readers.simple_sequence as ssr 
simple = ssr.SimpleSequence()
print simple.train
print simple.test


In [ ]:
for sequence in simple.train.seq_list: 
    print sequence
print "activity labels:"    
for sequence in simple.train.seq_list: 
    print sequence.x
# walk 0
# shop 1
# clean 2
print "weather labels:"    
for sequence in simple.train.seq_list: 
    print sequence.y
# sunny 1
# rainy 0

In [ ]:
import lxmls.sequences.hmm as hmmc
hmm = hmmc.HMM(simple.x_dict, simple.y_dict) 
hmm.train_supervised(simple.train, smoothing=0.1)
print "Initial Probabilities:", hmm.initial_probs
print "Initial counts:", hmm.initial_counts

print "Transition Probabilities:\n", hmm.transition_probs
print "transition counts:\n", hmm.transition_counts

print "Final Probabilities:", hmm.final_probs
print "final counts:", hmm.final_counts

print "Emission Probabilities\n", hmm.emission_probs
print "emission counts:\n", hmm.emission_counts


In [ ]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])

In [ ]:
print "Initial scores:\n", initial_scores
print "Transition scores:\n", transition_scores
print "Final scores:\n", final_scores
print "Emission scores:\n", emission_scores

In [ ]:
log_likelihood, forward = hmm.decoder.run_forward(initial_scores, transition_scores, final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood


In [ ]:
log_likelihood, backward = hmm.decoder.run_backward(initial_scores, transition_scores, final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood

In [ ]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])
state_posteriors, _, _ = hmm.compute_posteriors(initial_scores, transition_scores, final_scores, emission_scores)
 
print state_posteriors

In [ ]:
y_pred = hmm.posterior_decode(simple.test.seq_list[0]) 
print "Prediction test 0:\t", y_pred
print "Truth test 0:\t\t", simple.test.seq_list[0]

In [ ]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1]) 
print "Prediction test 1:\t", y_pred
print "Truth test 1:\t\t", simple.test.seq_list[1]

In [ ]:
import lxmls.readers.simple_sequence as ssr 
simple = ssr.SimpleSequence()

In [ ]:
import lxmls.sequences.hmm as hmmc
hmm = hmmc.HMM(simple.x_dict, simple.y_dict) 
hmm.train_supervised(simple.train, smoothing=0.1)
y_pred, score = hmm.viterbi_decode(simple.test.seq_list[0])
print "Viterbi decoding Prediction test 0 with smoothing:\n", y_pred, score
print "Truth test 0:", simple.test.seq_list[0]
y_pred, score = hmm.viterbi_decode(simple.test.seq_list[1])

print "Viterbi decoding Prediction test 1 with smoothing:\n", y_pred, score
print "Truth test 1:", simple.test.seq_list[1]

In [ ]:
import lxmls.readers.pos_corpus as pcc
%matplotlib inline
corpus = pcc.PostagCorpus()
train_seq = corpus.read_sequence_list_conll("data/train-02-21.conll",max_sent_len=15, max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll("data/test-23.conll",max_sent_len=15, max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll("data/dev-22.conll",max_sent_len=15,max_nr_sent=1000)
hmm = hmmc.HMM(corpus.word_dict, corpus.tag_dict)
hmm.train_supervised(train_seq)
hmm.print_transition_matrix()

In [ ]:
viterbi_pred_train = hmm.viterbi_decode_corpus(train_seq)
posterior_pred_train = hmm.posterior_decode_corpus(train_seq) 
eval_viterbi_train = hmm.evaluate_corpus(train_seq, viterbi_pred_train) 
eval_posterior_train = hmm.evaluate_corpus(train_seq, posterior_pred_train) 
print "Train Set Accuracy: Posterior Decode %.3f, Viterbi Decode: %.3f"%(eval_posterior_train,eval_viterbi_train)
viterbi_pred_test = hmm.viterbi_decode_corpus(test_seq) 
posterior_pred_test = hmm.posterior_decode_corpus(test_seq) 
eval_viterbi_test = hmm.evaluate_corpus(test_seq,viterbi_pred_test) 
eval_posterior_test = hmm.evaluate_corpus(test_seq,posterior_pred_test) 
print "Test Set Accuracy: Posterior Decode %.3f, Viterbi Decode: %.3f"%(eval_posterior_test,eval_viterbi_test)


In [ ]:
best_smoothing = hmm.pick_best_smoothing(train_seq, dev_seq, [10,1,0.1,0])

In [ ]:
hmm.train_supervised(train_seq, smoothing=best_smoothing)
viterbi_pred_test = hmm.viterbi_decode_corpus(test_seq)
posterior_pred_test = hmm.posterior_decode_corpus(test_seq)
eval_viterbi_test = hmm.evaluate_corpus(test_seq, viterbi_pred_test) 
eval_posterior_test = hmm.evaluate_corpus(test_seq, posterior_pred_test)
print "Best Smoothing %f -- Test Set Accuracy: Posterior Decode %.3f, Viterbi Decode: %.3f"%(
    best_smoothing,eval_posterior_test,eval_viterbi_test)

In [ ]:
import lxmls.sequences.confusion_matrix as cm
import matplotlib.pyplot as plt
confusion_matrix = cm.build_confusion_matrix(
    test_seq.seq_list, viterbi_pred_test, len(corpus.tag_dict), hmm.get_num_states())
cm.plot_confusion_bar_graph(confusion_matrix, corpus.tag_dict, xrange(hmm.get_num_states()), 'Confusion matrix')
plt.show()

# Day 4

In [ ]:
import cProfile
import numpy as np
%matplotlib inline

In [ ]:
import lxmls.sequences.crf_online as crfo
import lxmls.sequences.structured_perceptron as spc 
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc

In [ ]:
print "CRF Exercise"
corpus = pcc.PostagCorpus()
train_seq = corpus.read_sequence_list_conll("data/train-02-21.conll",max_sent_len=10, max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll("data/test-23.conll",max_sent_len=10, max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll("data/dev-22.conll",max_sent_len=10,max_nr_sent=1000)
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()

In [ ]:
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

In [ ]:
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print "CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train, eval_dev, eval_test)

In [ ]:
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

In [ ]:
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print "CRF - Extended Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train, eval_dev,eval_test)

In [ ]:
for i in xrange(50):
    if ((train_seq[i].y != pred_train[i].y).any()):
        print train_seq[i]
        print pred_train[i]

In [ ]:
for i in xrange(20):
    if ((dev_seq[i].y != pred_dev[i].y).any()):
        print dev_seq[i]
        print pred_dev[i]

In [ ]:
for i in xrange(20):
    if ((test_seq[i].y != pred_test[i].y).any()):
        print test_seq[i]
        print pred_test[i]

In [ ]:
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()
print "Perceptron Exercise"
sp = spc.StructuredPerceptron(corpus.word_dict, corpus.tag_dict, feature_mapper)
sp.num_epochs = 20
sp.train_supervised(train_seq)

In [ ]:
pred_train = sp.viterbi_decode_corpus(train_seq)
pred_dev = sp.viterbi_decode_corpus(dev_seq)
pred_test = sp.viterbi_decode_corpus(test_seq)
eval_train = sp.evaluate_corpus(train_seq, pred_train)
eval_dev = sp.evaluate_corpus(dev_seq, pred_dev)
eval_test = sp.evaluate_corpus(test_seq, pred_test)
print "Structured Perceptron - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%( eval_train,eval_dev,eval_test)

In [ ]:
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()
sp = spc.StructuredPerceptron(corpus.word_dict, corpus.tag_dict, feature_mapper)
sp.num_epochs = 20
sp.train_supervised(train_seq)

In [ ]:
pred_train = sp.viterbi_decode_corpus(train_seq)
pred_dev = sp.viterbi_decode_corpus(dev_seq)
pred_test = sp.viterbi_decode_corpus(test_seq)
eval_train = sp.evaluate_corpus(train_seq, pred_train)
eval_dev = sp.evaluate_corpus(dev_seq, pred_dev)
eval_test = sp.evaluate_corpus(test_seq, pred_test)
print "Structured Perceptron - Extended Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(
    eval_train,eval_dev,eval_test)

# Day 5

In [ ]:
import cProfile
import numpy as np
%matplotlib inline

In [ ]:
import sys 
sys.path.append('.' )
import lxmls.parsing.dependency_parser as depp
dp = depp.DependencyParser()
dp.read_data("portuguese")

In [ ]:
dp.train_perceptron(10)
dp.test()

In [ ]:
dp.features.use_lexical = True 
dp.read_data("portuguese") 
dp.train_perceptron(10) 
dp.test()
dp.features.use_distance = True 
dp.read_data("portuguese") 
dp.train_perceptron(10) 
dp.test()
dp.features.use_contextual = True 
dp.read_data("portuguese") 
dp.train_perceptron(10)
dp.test()

In [ ]:
dp.train_crf_sgd(10, 0.01, 0.1)
dp.test()

In [ ]:
dp.read_data("english")
dp.train_perceptron(10)
dp.test()

In [ ]:
dp = depp.DependencyParser() 
dp.features.use_lexical = True 
dp.features.use_distance = True 
dp.features.use_contextual = True 
dp.read_data("english") 
dp.projective = True
dp.train_perceptron(10)
dp.test()

# Day 6

In [ ]:
import cProfile
import numpy as np
%matplotlib inline

In [ ]:
import lxmls.readers.sentiment_reader as srs 
scr = srs.SentimentCorpus("books")
train_x = scr.train_X.T
train_y = scr.train_y[:, 0]
test_x = scr.test_X.T
test_y = scr.test_y[:, 0]

In [ ]:
# Neural network modules
import lxmls.deep_learning.mlp as dl 
import lxmls.deep_learning.sgd as sgd # Model parameters
geometry = [train_x.shape[0], 20, 2] 
actvfunc = ['sigmoid',  'softmax']
# Instantiate model
mlp = dl.NumpyMLP(geometry, actvfunc)

In [ ]:
# Model parameters
n_iter = 5
bsize  = 5
lrate  = 0.01
# Train
sgd.SGD_train(mlp, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y))
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]
print "MLP (%s) Amazon Sentiment Accuracy train: %f test: %f" % (geometry, acc_train, acc_test)

## Theano

In [ ]:
x = test_x             # Test set
W1, b1 = mlp.params[:2]     # Weights and bias of fist layer
z1= np.dot(W1, x) + b1 # Linear transformation
tilde_z1 = 1/(1+np.exp(-z1))  # Non-linear transformation

In [ ]:
# Theano code.
# NOTE: We use undescore to denote symbolic equivalents to Numpy variables. # This is no Python convention!.
import theano
import theano.tensor as T
_x = T.matrix('x')

In [ ]:
_W1 = theano.shared(value=W1, name='W1', borrow=True)
_b1 = theano.shared(value=b1, name='b1', borrow=True, broadcastable=(False, True))

In [ ]:
_z1            = T.dot(_W1, _x) + _b1
_tilde_z1      = T.nnet.sigmoid(_z1)
# Keep in mind that naming variables is useful when debugging
_z1.name       = 'z1'
_tilde_z1.name = 'tilde_z1'

In [ ]:
# Perceptron computation graph
theano.printing.debugprint(_tilde_z1)

In [ ]:
layer1 = theano.function([_x], _tilde_z1)

In [ ]:
# Check Numpy and Theano match
np.allclose(tilde_z1, layer1(x.astype(theano.config.floatX)))

In [ ]:
train_x = train_x.astype(theano.config.floatX)
train_y = train_y.astype('int32')

In [ ]:
mlp_a = dl.NumpyMLP(geometry, actvfunc)
mlp_b = dl.TheanoMLP(geometry, actvfunc)

In [ ]:
np.allclose(mlp_a.forward(test_x), mlp_b.forward(test_x))

In [ ]:
W2, b2 = mlp.params[2:] # Weights and bias of second (and last!) layer
# Second layer symbolic variables
_W2 = theano.shared(value=W2, name='W2', borrow=True)
_b2 = theano.shared(value=b2, name='b2', borrow=True, broadcastable=(False, True)) # Second layer symbolic expressions
_z2       = T.dot(_W2, _tilde_z1) + _b2
_tilde_z2 = T.nnet.softmax(_z2.T).T

In [ ]:
_y = T.ivector('y')
_F = -T.mean(T.log(_tilde_z2[_y, T.arange(_y.shape[0])]))

In [ ]:
_nabla_F = T.grad(_F, _W1)
nabla_F = theano.function([_x, _y], _nabla_F)

In [ ]:
mlp_c   = dl.TheanoMLP(geometry, actvfunc)
_x      = T.matrix('x')
_y      = T.ivector('y')
_F      = mlp_c._cost(_x, _y)
# SGD update rule
updates = [(par, par - lrate*T.grad(_F, par)) for par in mlp_c.params]

In [ ]:
batch_up = theano.function([_x, _y], _F, updates=updates)
n_batch = int(np.ceil(float(train_x.shape[1])/bsize))

In [ ]:
import time
# Model
geometry = [train_x.shape[0], 20, 2] 
actvfunc = ['sigmoid', 'softmax']
# Numpy MLP
mlp_a = dl.NumpyMLP(geometry, actvfunc)
init_t = time.clock()
sgd.SGD_train(mlp_a, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y)) 
print "\nNumpy version took %2.2f sec" % (time.clock() - init_t)
acc_train = sgd.class_acc(mlp_a.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp_a.forward(test_x), test_y)[0]
print "Amazon Sentiment Accuracy train: %f test: %f\n" % (acc_train, acc_test)
# Theano grads
mlp_b = dl.TheanoMLP(geometry, actvfunc)
init_t = time.clock()
sgd.SGD_train(mlp_b, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y)) 
print "\nCompiled gradient version took %2.2f sec" % (time.clock() - init_t) 
acc_train = sgd.class_acc(mlp_b.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp_b.forward(test_x), test_y)[0]
print "Amazon Sentiment Accuracy train: %f test: %f\n" % (acc_train, acc_test)
# Theano batch update
init_t = time.clock()
sgd.SGD_train(mlp_c, n_iter, batch_up=batch_up, n_batch=n_batch, bsize=bsize,
train_set=(train_x, train_y))
print "\nTheano compiled batch update version took %2.2f" % (time.clock() - init_t) 
acc_train = sgd.class_acc(mlp_c.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp_c.forward(test_x), test_y)[0]
print "Amazon Sentiment Accuracy train: %f test: %f\n"%(acc_train,acc_test)

# Day 7

In [ ]:
# Load Part-of-Speech data
import lxmls.readers.pos_corpus as pcc
corpus = pcc.PostagCorpus()
train_seq = corpus.read_sequence_list_conll("data/train-02-21.conll", max_sent_len=15,
    max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll("data/test-23.conll", max_sent_len=15,
    max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll("data/dev-22.conll", max_sent_len=15,
    max_nr_sent=1000)

In [ ]:
# Redo indices
train_seq, test_seq, dev_seq = pcc.compacify(train_seq, test_seq, dev_seq, theano=True) 
# Get number of words and tags in the corpus
nr_words = len(train_seq.x_dict)
nr_tags = len(train_seq.y_dict)

In [ ]:
import lxmls.deep_learning.rnn as rnns
#reload(rnns)
# RNN configuration
SEED = 1234 # Random seed to initialize weigths 
emb_size = 50 # Size of word embeddings 
hidden_size = 20 # size of hidden layer
# RNN
np_rnn = rnns.NumpyRNN(nr_words, emb_size, hidden_size, nr_tags, seed=SEED)
# Example sentence
x0 = train_seq[0].x
y0 = train_seq[0].y

In [ ]:
# Forward pass
p_y, y_rnn, h, z1, x = np_rnn.forward(x0, all_outputs=True) 
# Gradients
numpy_rnn_gradients = np_rnn.grads(x0, y0)

In [ ]:
import numpy as np
import theano
import theano.tensor as T 
theano.config.optimizer='None'
def square(x): 
    return x**2
# Python
def np_square_n_steps(nr_steps): 
    out = []
    for n in np.arange(nr_steps): 
        out.append(square(n))
    return np.array(out)

# Theano
nr_steps = T.lscalar('nr_steps')
h, _ = theano.scan(fn=square, sequences=T.arange(nr_steps))
th_square_n_steps = theano.function([nr_steps], h)
# Compare both
print np_square_n_steps(15) 
print th_square_n_steps(13)

In [ ]:
# Configuration
nr_states = 3
nr_steps = 5
# Transition matrix
A = np.abs(np.random.randn(nr_states, nr_states)) 
A = A/A.sum(0, keepdims=True)
# Initial state
s0 = np.zeros(nr_states)
s0[0] = 1

In [ ]:
# Numpy version
def np_markov_step(s_tm1): 
    s_t = np.dot(s_tm1, A.T) 
    return s_t
def np_markov_chain(nr_steps, A, s0):
# Pre-allocate space
    s = np.zeros((nr_steps+1, nr_states)) 
    s[0, :] = s0
    for t in np.arange(nr_steps):
        s[t+1, :] = np_markov_step(s[t, :]) 
    return s
np_markov_chain(nr_steps, A, s0)

In [ ]:
# Theano version
# Store variables as shared variables
th_A = theano.shared(A, name='A', borrow=True) 
th_s0 = theano.shared(s0, name='s0', borrow=True) 
# Symbolic variable for the number of steps 
th_nr_steps = T.lscalar('nr_steps')
def th_markov_step(s_tm1):
    s_t = T.dot(s_tm1, th_A.T)
    # Remember to name variables 
    s_t.name = 's_t'
    return s_t
s, _ = theano.scan(th_markov_step, outputs_info=[dict(initial=th_s0)],n_steps=th_nr_steps)
th_markov_chain = theano.function([th_nr_steps], T.concatenate((th_s0[None, :], s), 0))
th_markov_chain(nr_steps)

In [ ]:
# Instantiate the class
rnn = rnns.RNN(nr_words, emb_size, hidden_size, nr_tags, seed=SEED)
# Compile the forward pass function
x = T.ivector('x')
th_forward = theano.function([x], rnn._forward(x).T)

In [ ]:
assert np.allclose(th_forward(x0), np_rnn.forward(x0)), "Numpy and Theano forward pass differ!"

In [ ]:
# Compile function returning the list of gradients
x = T.ivector('x')     # Input words
y = T.ivector('y')     # gold tags
p_y = rnn._forward(x)
cost = -T.mean(T.log(p_y)[T.arange(y.shape[0]), y])
grads_fun = theano.function([x, y], [T.grad(cost, par) for par in rnn.param])

In [ ]:
# Compare numpy and theano gradients
theano_rnn_gradients = grads_fun(x0, y0) 
for n in range(len(theano_rnn_gradients)):
    assert np.allclose(numpy_rnn_gradients[n], theano_rnn_gradients[n]), "Numpy and Theano gradients differ in step n"

In [ ]:
# Parameters
lrate = 0.5 # Learning rate
n_iter = 5 # Number of iterations
# Get list of SGD batch update rule for each parameter
updates = [(par, par - lrate*T.grad(cost, par)) for par in rnn.param] # compile
rnn_prediction = theano.function([x], T.argmax(p_y, 1)) 
rnn_batch_update = theano.function([x, y], cost, updates=updates)

In [ ]:
import sys
nr_words = sum([len(seq.x) for seq in train_seq]) 
for i in range(n_iter):
    # Training
    cost = 0
    errors = 0
    for n, seq in enumerate(train_seq):
        cost += rnn_batch_update(seq.x, seq.y)
        errors += sum(rnn_prediction(seq.x) != seq.y)
    acc_train = 100*(1-errors*1./nr_words)
    print "Epoch %d: Train cost %2.2f Acc %2.2f %%" % (i+1, cost, acc_train), # Evaluation
    errors = 0
    for n, seq in enumerate(dev_seq):
        errors += sum(rnn_prediction(seq.x) != seq.y) 
    acc_dev = 100*(1-errors*1./nr_words)
    print " Devel Acc %2.2f %%" % acc_dev 
    sys.stdout.flush()

In [ ]:
# Embeddings Path
import lxmls.deep_learning.embeddings as emb 
import os
reload(emb)
if not os.path.isfile('EMBEDDINGS'):
    emb.download_embeddings('senna_50', "data/senna_50")
E = emb.extract_embeddings("data/senna_50", train_seq.x_dict)
# Reset model to remove the effect of training
rnn = rnns.reset_model(rnn, seed=SEED)
# Set the embedding layer to the pre-trained values
rnn.param[0].set_value(E.astype(theano.config.floatX))
# Now re-run SGD training

In [ ]:
# Instantiate LSTM
lstm = rnns.LSTM(nr_words, emb_size, hidden_size, nr_tags)
# Compile prediction and bacth update functions
lstm_prediction = theano.function([x], T.argmax(lstm._forward(x), 1))
lstm_cost = -T.mean(T.log(lstm._forward(x))[T.arange(y.shape[0]), y])
# Get list of SGD batch update rule for each parameter
lstm_updates = [(par, par - lrate*T.grad(lstm_cost, par)) for par in lstm.param] # compile
lstm_batch_update = theano.function([x, y], lstm_cost, updates=lstm_updates)

In [ ]:
nr_words = sum([len(seq.x) for seq in train_seq]) 
for i in range(n_iter):
    # Training
    cost = 0
    errors = 0
    for n, seq in enumerate(train_seq):
        cost += lstm_batch_update(seq.x, seq.y)
        errors += sum(lstm_prediction(seq.x) != seq.y)
    acc_train = 100*(1-errors*1./nr_words)
    print "Epoch %d: Train cost %2.2f Acc %2.2f %%" % (i+1, cost, acc_train), 
    # Evaluation:
    errors = 0
    for n, seq in enumerate(dev_seq):
        errors += sum(lstm_prediction(seq.x) != seq.y) 
        acc_dev = 100*(1-errors*1./nr_words)
    print " Devel Acc %2.2f %%" % acc_dev 
    sys.stdout.flush()
                 